# Timer

In [5]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 21, 7, 0

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [6]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = True

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [20, 25, 30, 35]},
              'STOCH': {'fastk_period': [3, 5, 7, 9], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [2, 3, 5, 7], 'low_bound': [10, 15, 20, 25]}}
# optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [35]},
#                   'STOCH': {'fastk_period': [6], 'slowk_period': [2],
#                             'slowd_period': [2], 'low_bound': [25]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 3072


  0%|          | 0/3072 [00:00<?, ?it/s]


Load the datasets...
Binance
BinanceFutures


,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
114,STOCH_RSI,12,20,5,5,2,20,96.88,87.50,84.38,75.00,78.12,84.38,84.38,68.75,59.38,59.38,62.50,65.62,71.88,68.75,71.88,71.88,78.12,78.12,81.25,81.25,84.38,87.50,84.38,87.50,0.38,0.48,0.44,0.29,0.45,0.48,0.53,0.38,0.49,0.32,0.23,0.36,0.46,0.35,0.24,0.32,0.38,0.92,1.06,1.23,0.95,1.38,1.17,1.23,32,77.215000,230.880000
74,STOCH_RSI,12,20,5,2,5,20,95.00,80.00,80.00,70.00,75.00,80.00,85.00,80.00,70.00,65.00,75.00,75.00,75.00,75.00,80.00,80.00,75.00,90.00,95.00,90.00,95.00,90.00,90.00,90.00,0.36,0.39,0.46,0.29,0.28,0.46,0.57,0.46,0.51,0.46,0.44,0.49,0.46,0.30,0.51,0.36,0.33,1.02,1.08,1.34,1.13,1.63,1.14,1.17,20,81.458333,229.166667
85,STOCH_RSI,12,20,5,3,3,15,87.50,87.50,93.75,75.00,75.00,87.50,87.50,87.50,75.00,81.25,75.00,81.25,81.25,81.25,81.25,75.00,68.75,87.50,87.50,87.50,93.75,93.75,93.75,93.75,0.38,0.47,0.47,0.50,0.43,0.69,0.65,0.52,0.57,0.44,0.39,0.46,0.43,0.57,0.63,0.53,0.34,0.87,0.99,1.14,1.30,1.67,1.43,1.22,16,84.114583,225.833333
98,STOCH_RSI,12,20,5,4,2,20,91.89,86.49,81.08,70.27,72.97,81.08,78.38,70.27,64.86,70.27,67.57,72.97,70.27,64.86,67.57,67.57,67.57,70.27,70.27,75.68,81.08,86.49,86.49,86.49,0.36,0.32,0.41,0.26,0.41,0.54,0.53,0.46,0.43,0.37,0.34,0.33,0.42,0.45,0.57,0.45,0.32,0.95,1.06,1.25,1.09,1.53,1.25,1.30,37,75.112917,189.177917
240,STOCH_RSI,12,20,9,5,2,10,100.00,90.00,90.00,100.00,100.00,100.00,90.00,80.00,60.00,60.00,80.00,80.00,90.00,100.00,100.00,90.00,100.00,100.00,90.00,100.00,100.00,90.00,70.00,70.00,0.46,0.68,0.59,0.65,0.75,0.76,0.57,0.42,0.42,0.25,0.30,0.42,0.55,0.49,0.31,0.40,0.52,0.49,0.55,0.45,0.36,0.54,0.54,0.69,10,88.750000,187.500000
1035,STOCH_RSI,14,20,3,2,5,25,100.00,100.00,100.00,77.78,88.89,77.78,100.00,88.89,77.78,77.78,88.89,77.78,88.89,88.89,88.89,88.89,88.89,88.89,100.00,100.00,100.00,100.00,88.89,100.00,0.69,0.65,0.48,0.87,0.79,0.54,0.82,0.48,0.54,0.26,0.37,0.46,1.13,0.73,0.80,0.86,0.75,1.20,1.20,1.66,1.67,2.31,1.61,1.93,9,90.741667,186.675000
151,STOCH_RSI,12,20,7,3,3,25,90.00,82.50,82.50,75.00,77.50,77.50,77.50,67.50,62.50,72.50,72.50,75.00,67.50,65.00,67.50,67.50,65.00,67.50,70.00,75.00,77.50,87.50,85.00,82.50,0.35,0.32,0.46,0.41,0.46,0.57,0.57,0.47,0.51,0.48,0.50,0.48,0.38,0.74,0.75,0.71,0.45,0.85,0.99,0.97,1.70,1.73,1.39,1.41,40,74.583333,183.333333
99,STOCH_RSI,12,20,5,4,2,25,88.64,84.09,84.09,72.73,75.00,79.55,79.55,72.73,65.91,70.45,68.18,72.73,65.91,63.64,68.18,68.18,63.64,65.91,65.91,72.73,77.27,86.36,84.09,84.09,0.36,0.32,0.37,0.26,0.38,0.43,0.53,0.47,0.45,0.38,0.35,0.35,0.40,0.41,0.63,0.40,0.29,0.92,0.98,1.14,0.79,1.17,1.17,1.28,44,74.148333,182.526667
113,STOCH_RSI,12,20,5,5,2,15,95.45,90.91,86.36,72.73,81.82,86.36,86.36,63.64,50.00,59.09,68.18,72.73,77.27,68.18,68.18,68.18,77.27,77.27,81.82,81.82,86.36,90.91,86.36,90.91,0.42,0.47,0.41,0.29,0.47,0.48,0.42,0.38,0.24,0.23,0.23,0.36,0.46,0.30,0.18,0.25,0.29,0.48,0.56,0.90,0.92,1.17,1.17,1.23,22,77.840000,172.480000
1121,STOCH_RSI,14,20,5,4,2,

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [12], 'low_bound': [20]},
                  'STOCH': {'fastk_period': [9], 'slowk_period': [2],
                            'slowd_period': [7], 'low_bound': [15]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)